In [1]:
import sys
sys.path.insert(0, '../Utilities/')

import torch
from collections import OrderedDict

from doe_lhs import *
import numpy as np
import matplotlib.pyplot as plt
import scipy.io
from scipy.interpolate import griddata
# from plotting import newfig, savefig
from mpl_toolkits.axes_grid1 import make_axes_locatable
import matplotlib.gridspec as gridspec
import time
from torch.utils.data import TensorDataset, DataLoader

np.random.seed(1234)

In [2]:
# CUDA support 
if torch.cuda.is_available():
    device = torch.device('cuda')
else:
    device = torch.device('cpu')
device

device(type='cuda')

In [3]:
# the deep neural network
class DNN(torch.nn.Module):
    def __init__(self, layers):
        super(DNN, self).__init__()
        
        # parameters
        self.depth = len(layers) - 1
        
        # set up layer order dict
        self.activation = torch.nn.Tanh
        
        layer_list = list()
        for i in range(self.depth - 1): 
            layer_list.append(
                ('layer_%d' % i, torch.nn.Linear(layers[i], layers[i+1]))
            )
            layer_list.append(('activation_%d' % i, self.activation()))
            
        layer_list.append(
            ('layer_%d' % (self.depth - 1), torch.nn.Linear(layers[-2], layers[-1]))
        )
        layerDict = OrderedDict(layer_list)
        
        # deploy layers
        self.layers = torch.nn.Sequential(layerDict)
        
    def forward(self, x):
        out = self.layers(x)
        return out

In [4]:
# the physics-guided neural network
class PhysicsInformedNN():
    def __init__(self,x_u, y_u, x_f,x_lb,x_ub,X_u_ts ,u_ts , net, device,phase1):
        super(PhysicsInformedNN, self).__init__()
        
        self.batch_size = 16
        self.device = device
        shuffle = True
        
        if phase1:
            self.train_x_u = torch.tensor(x_u, requires_grad=True).float().to(self.device)
            self.train_y_u = torch.tensor(y_u, requires_grad=True).float().to(self.device)
            self.train_x_f = torch.tensor(x_f, requires_grad=True).float().to(self.device)
            
            self.train_x_lb = torch.tensor(x_lb, requires_grad=True).float().to(device)
            self.train_x_ub = torch.tensor(x_ub, requires_grad=True).float().to(device) 
            
        else:            
            self.train_x_u = torch.tensor(x_u, requires_grad=True).float().to(self.device)
            self.train_y_u = torch.tensor(y_u, requires_grad=True).float().to(self.device)
            self.train_x_f = torch.tensor(x_f, requires_grad=True).float().to(self.device)
            
            self.x_u_ts = torch.tensor(X_u_ts[1:-1, :], requires_grad=True).float().to(device)
            self.u_ts = torch.tensor(u_ts[1:-1]).float().to(device)
            u_weights = torch.rand((510, 1))
            self.u_weights = torch.tensor(u_weights, requires_grad=True)
            
            self.train_x_lb = torch.tensor(x_lb, requires_grad=True).float().to(device)
            self.train_x_ub = torch.tensor(x_ub, requires_grad=True).float().to(device)
            
            self.Adam_optim_u = torch.optim.Adam(
                [self.u_weights],
                lr = 5e-3
            )
    
        # deep neural networks
        self.dnn = net

        # Adam optimizer
        self.Adam_optim = torch.optim.Adam(
            self.dnn.parameters(),
            lr = 1e-4,
            betas = (0.9, 0.999)
        )
        # L-BFGS optimizer
        self.optimizer = torch.optim.LBFGS(
            self.dnn.parameters(), 
            lr=1.0, 
            max_iter=50000, 
            max_eval=50000, 
            history_size=50,
            tolerance_grad=1e-5, 
            tolerance_change=1e-9,
            line_search_fn="strong_wolfe"       # can be "strong_wolfe"
        )
        
        self.train_loader = DataLoader(
            list(zip(self.train_x_u,self.train_y_u)), batch_size=self.batch_size, shuffle=shuffle
        )
        
        self.iter = 0
    
    def get_residual(self,X):
        x = torch.tensor(X[:, 0:1], requires_grad=True).float().to(self.device)
        t = torch.tensor(X[:, 1:2], requires_grad=True).float().to(self.device)

        u = self.net_u(x, t)
        f = self.net_f(x, t, u)
        return u, f
    
    
    def net_u(self, x, t):  
        u = self.dnn(torch.cat([x, t], dim=1))
        return u
    
    def boundary_loss(self):
        x_lb = torch.tensor(self.train_x_lb[:, 0:1], requires_grad=True).float().to(self.device)
        t_lb = torch.tensor(self.train_x_lb[:, 1:2], requires_grad=True).float().to(self.device)

        x_ub = torch.tensor(self.train_x_ub[:, 0:1], requires_grad=True).float().to(self.device)
        t_ub = torch.tensor(self.train_x_ub[:, 1:2], requires_grad=True).float().to(self.device)
        
        y_pred_lb = self.dnn.forward(torch.cat([x_lb, t_lb], dim=1))
        y_pred_ub = self.dnn.forward(torch.cat([x_ub, t_ub], dim=1))
        
        u_lb = y_pred_lb[:,0:1]
        u_ub = y_pred_ub[:,0:1]

        u_lb_x = torch.autograd.grad(
                u_lb, x_lb, 
                grad_outputs=torch.ones_like(u_lb),
                retain_graph=True,
                create_graph=True
            )[0]

        u_ub_x = torch.autograd.grad(
                u_ub, x_ub, 
                grad_outputs=torch.ones_like(u_ub),
                retain_graph=True,
                create_graph=True
            )[0]

        loss = torch.mean((y_pred_lb - y_pred_ub)**2) + \
               torch.mean((u_lb_x - u_ub_x)**2)
        return loss
    
    
    def net_f(self, x, t, u):
        """ The pytorch autograd version of calculating residual """
        
        u_t = torch.autograd.grad(
            u, t, 
            grad_outputs=torch.ones_like(u),
            retain_graph=True,
            create_graph=True
        )[0]
        u_x = torch.autograd.grad(
            u, x, 
            grad_outputs=torch.ones_like(u),
            retain_graph=True,
            create_graph=True
        )[0]
        u_xx = torch.autograd.grad(
            u_x, x, 
            grad_outputs=torch.ones_like(u_x),
            retain_graph=True,
            create_graph=True
        )[0]

        f = u_t - 0.0001* u_xx  + 5*u*u*u - 5*u 
        return f
    
    def loss_func(self):
        
        if self.phase1:
            self.optimizer.zero_grad()
            x = self.train_x_u
            y = self.train_y_u
            
            y_pred, _ = self.get_residual(x)
            _, f_u  = self.get_residual(self.train_x_f)
            
            loss_u = torch.mean((y - y_pred) ** 2)
            loss_f = torch.mean(f_u ** 2)
            
            b_loss = self.boundary_loss()

            loss = 100 * loss_u + loss_f + b_loss
            loss.backward()
            
            self.iter += 1
#             if self.iter % 100 == 0:
#                 print(
#                     'Iter %d, Loss: %.5e, Loss_u: %.5e, Loss_f: %.5e' % (self.iter, loss.item(), loss_u.item(), loss_f.item())
#                 )
            if self.iter % 100 == 0:
                u_pred, f_pred = model.predict(self.X)
                error_u = np.linalg.norm(self.U-u_pred,2)/np.linalg.norm(self.U,2)
                print('Iter %d, Error u: %e' % (self.iter, error_u)) 
            return loss
        else:
            
            self.optimizer.zero_grad()
            x = self.train_x_u
            y = self.train_y_u

            y_pred, _ = self.get_residual(x)
            _, f_u  = self.get_residual(self.train_x_f)
            
            u_ts_pred, f_ts_pred = self.get_residual(self.x_u_ts)
            
            loss_u = torch.mean((y - y_pred) ** 2)
            loss_f = torch.mean(f_u ** 2)

            b_loss_1 = self.boundary_loss()
            
#             loss_u_ts = torch.mean((self.u_ts - (self.u_weights.float().to(device) * u_ts_pred))**2)
            loss_u_ts = torch.mean((self.u_ts -  u_ts_pred)**2)
            loss_f_ts = torch.mean(f_ts_pred ** 2)

    
            loss = 100 * loss_u + loss_f + loss_u_ts + loss_f_ts + b_loss_1
            loss.backward()
            self.iter += 1
#             if self.iter % 100 == 0:
#                 print(
#                     ' Loss: %.5e, Loss_u: %.5e, Loss_f: %.5e, loss_u_ts: %.5e, loss_f_ts: %.5e' % (loss.item(), loss_u.item(), loss_f.item(), loss_u_ts.item(), loss_f_ts.item())
#                 )
            if self.iter % 100 == 0:
                u_pred, f_pred = model.predict(self.X)
                error_u = np.linalg.norm(self.U-u_pred,2)/np.linalg.norm(self.U,2)
                print('Iter %d, Error u: %e' % (self.iter, error_u)) 
                
            return loss
    
    def train(self, phase_1, epoch,X,U):
        if phase_1:
            
#             for i in range(int(epoch / 2000)):
            self.train_1(True,epoch,X,U)
            self.train_2(True)
        else:
#             for i in range(int(epoch / 2000)):
            self.train_1(False,epoch,X,U)
            self.train_2(False)
    
    def train_1(self,phase_1,epoch,X,U):
        
        self.X = X
        self.U = U
        
        if phase_1:
            for e in range(epoch):
                for i, (x, y) in enumerate(self.train_loader):
                    self.dnn.train()
                    self.optimizer.zero_grad()

                    y_pred, _ = self.get_residual(x)
                    _, f_u  = self.get_residual(self.train_x_f)
                    
                    loss_u = torch.mean((y - y_pred) ** 2)
                    loss_f = torch.mean(f_u ** 2)

                    b_loss = self.boundary_loss()
                    
                    loss = 100 * loss_u + loss_f + b_loss
                    loss.backward(retain_graph = True)
                    self.Adam_optim.step()

                self.iter += 1
                if self.iter % 100 == 0:
                    u_pred, f_pred = model.predict(X)
                    error_u = np.linalg.norm(U-u_pred,2)/np.linalg.norm(U,2)
                    print('Iter %d, Error u: %e' % (self.iter, error_u)) 

#                     print(
#                         'Iter %d, Loss: %.5e, Loss_u: %.5e, Loss_f: %.5e' % (self.iter, loss.item(), loss_u.item(), loss_f.item())
#                     )
        
        else:
            for e in range(epoch):  
                for i, (x, y) in enumerate(self.train_loader):
                    self.dnn.train()
                    self.optimizer.zero_grad()
                    self.Adam_optim_u.zero_grad()

                    y_pred, _ = self.get_residual(x)
                    _, f_u  = self.get_residual(self.train_x_f)

                    u_ts_pred, f_ts_pred = self.get_residual(self.x_u_ts)
                    
                    loss_u = torch.mean((y - y_pred) ** 2)
                    loss_f = torch.mean(f_u ** 2)
        
                    b_loss_1 = self.boundary_loss()
        
                    loss_u_ts = torch.mean((self.u_ts - (self.u_weights.float().to(device) * u_ts_pred))**2)
#                     loss_u_ts = torch.mean((self.u_ts -  u_ts_pred)**2)
                    loss_f_ts = torch.mean(f_ts_pred ** 2)

                    loss = 100 * loss_u + loss_f +  loss_u_ts + loss_f_ts + b_loss_1
                    loss.backward(retain_graph = True)
                    self.Adam_optim.step()
                    self.Adam_optim_u.step()
                    
                    
                self.iter += 1
                if self.iter % 100 == 0:
                    u_pred, f_pred = model.predict(X)
                    error_u = np.linalg.norm(U-u_pred,2)/np.linalg.norm(U,2)
                    print('Iter %d, Error u: %e' % (self.iter, error_u)) 

    def train_2(self,phase1):
        self.dnn.train()
        self.phase1 = phase1
        self.optimizer.step(self.loss_func)
            
    def predict(self, X):
        x = torch.tensor(X[:, 0:1], requires_grad=True).float().to(device)
        t = torch.tensor(X[:, 1:2], requires_grad=True).float().to(device)

        self.dnn.eval()
        u = self.net_u(x, t)
        f = self.net_f(x, t, u)
        u = u.detach().cpu().numpy()
        f = f.detach().cpu().numpy()
        return u, f

In [5]:


N_u = 200
N_i = 512
N_f = 10000

# N_u = 200
# N_i = 100
# N_f = 20000

layers = [2, 128, 128, 128, 128, 1]

data = scipy.io.loadmat('AC.mat')

t = data['tt'].flatten()[:,None] # (201,1)
x = data['x'].flatten()[:,None]  # (512,1)
Exact = np.real(data['uu'])

X, T = np.meshgrid(x,t)

X_star = np.hstack((X.flatten()[:,None], T.flatten()[:,None]))
u_star = Exact.T.flatten()[:,None]              


st = 0.30
tt = int(t.shape[0] * st)

trunk1_X = X_star[ : x.shape[0] * tt]
trunk2_X = X_star[x.shape[0] * (tt - 1) : ]

trunk1_u = u_star[ : x.shape[0] * tt]
trunk2_u = u_star[x.shape[0] * (tt - 1) : ]


# Doman bounds
first_lb = trunk1_X.min(0) # [-1.  0.] 
first_ub = trunk1_X.max(0) # [1.   0.49]

second_lb = trunk2_X.min(0) # [-1.    0.49]
second_ub = trunk2_X.max(0) # [1.   0.99]  



# --------------------- first half ---------------------


# initial points
idx_x_1 = np.random.choice(x.shape[0], int(N_i), replace=False)
X_i_train_1 = x[idx_x_1,:]
u_i_train_1 = Exact[idx_x_1,0:1]

# boundary points
idx_t = np.random.choice(tt, int(N_u), replace=True)
tb_1 = t[idx_t,:]

# collocation points
X_f_1 = first_lb + (first_ub-first_lb)*lhs(2, int(N_f))


X0_1 = np.concatenate((X_i_train_1, 0*X_i_train_1), 1) # (x0, 0)
Y0_1 = u_i_train_1 
X_lb_1 = np.concatenate((0*tb_1 + first_lb[0], tb_1), 1) # (lb[0], tb)
X_ub_1 = np.concatenate((0*tb_1 + first_ub[0], tb_1), 1) # (ub[0], tb)

net = DNN(layers).to(device)
model = PhysicsInformedNN(X0_1, Y0_1, X_f_1, X_lb_1, X_ub_1,[],[], net, device,True)

In [8]:
%%time
model.train(True,epoch = 100,X = trunk1_X,U = trunk1_u)

u_pred, f_pred = model.predict(trunk1_X)
error_u = np.linalg.norm(trunk1_u-u_pred,2)/np.linalg.norm(trunk1_u,2)
print('Error u: %e' % (error_u)) 

/tmp/dcs-tmp.u1834031/ipykernel_2033954/372614191.py:64: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  x = torch.tensor(X[:, 0:1], requires_grad=True).float().to(self.device)
/tmp/dcs-tmp.u1834031/ipykernel_2033954/372614191.py:65: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  t = torch.tensor(X[:, 1:2], requires_grad=True).float().to(self.device)
/tmp/dcs-tmp.u1834031/ipykernel_2033954/372614191.py:77: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  x_lb = torch.tensor(self.train_x_lb[:, 0:1], requires_grad=True).float().to(self.device)
/t

Iter 8400, Error u: 8.017442e-03
Iter 8500, Error u: 2.540645e-03
Iter 8600, Error u: 2.523608e-03
Iter 8700, Error u: 2.850600e-03
Iter 8800, Error u: 2.785782e-03
Iter 8900, Error u: 2.852900e-03
Iter 9000, Error u: 2.874274e-03
Iter 9100, Error u: 2.986973e-03
Iter 9200, Error u: 2.830830e-03
Iter 9300, Error u: 2.679850e-03
Iter 9400, Error u: 2.730910e-03
Iter 9500, Error u: 2.736483e-03
Iter 9600, Error u: 2.702561e-03
Iter 9700, Error u: 2.430586e-03
Iter 9800, Error u: 2.472736e-03
Iter 9900, Error u: 2.530494e-03
Iter 10000, Error u: 2.497077e-03
Iter 10100, Error u: 2.528688e-03
Iter 10200, Error u: 2.411789e-03
Iter 10300, Error u: 2.362256e-03
Iter 10400, Error u: 2.276402e-03
Iter 10500, Error u: 2.118370e-03
Iter 10600, Error u: 2.030640e-03
Iter 10700, Error u: 2.024045e-03
Iter 10800, Error u: 1.952830e-03
Iter 10900, Error u: 2.000107e-03
Iter 11000, Error u: 2.001488e-03
Iter 11100, Error u: 1.897493e-03
Iter 11200, Error u: 1.849394e-03
Iter 11300, Error u: 1.630368e

In [9]:
u_pred, f_pred = model.predict(trunk1_X)
error_u = np.linalg.norm(trunk1_u-u_pred,2)/np.linalg.norm(trunk1_u,2)
print('Error u: %e' % (error_u))

Error u: 1.624384e-03


In [10]:
# --------------------- second half ---------------------


tmp_pred = u_pred[-512:]

# initial points

idx_x = np.random.choice(x.shape[0], 512, replace=False)
X0_2 = np.concatenate((x[idx_x,:], np.full(x[idx_x,:].shape, first_ub[1])), 1) # (x0, 0)
Y0_2 = u_pred[-512:][idx_x,0:1]


# boundary points
# idx_t = np.random.choice(range(tt, 201), int(N_u - N_u * st), replace=False)
idx_t = np.random.choice(range(tt, 201), int(N_u - (N_u * 0.3) ), replace=True)
tb_2 = t[idx_t,:]

idx_t = np.random.choice(range(0, tt), int(N_u/2 ), replace=True)
tb_1 = t[idx_t,:]


# collocation points
X_f_train_1 = first_lb + (first_ub-first_lb)*lhs(2, int(N_f /2))
X_f_train_2 = second_lb + (second_ub-second_lb)*lhs(2, int(N_f - N_f*0.3 ))

X_lb_1 = np.concatenate((0*tb_1 + first_lb[0], tb_1), 1) # (lb[0], tb)
X_ub_1 = np.concatenate((0*tb_1 + first_ub[0], tb_1), 1) # (ub[0], tb)
X_lb_2 = np.concatenate((0*tb_2 + second_lb[0], tb_2), 1) # (lb[0], tb)
X_ub_2 = np.concatenate((0*tb_2 + second_ub[0], tb_2), 1) # (ub[0], tb)


model = PhysicsInformedNN(X0_1,
                       Y0_1,
                       np.vstack([X_f_train_1,X_f_train_2]), 
                       np.vstack([X_lb_1,X_lb_2]), 
                       np.vstack([X_ub_1,X_ub_2]), 
                       X0_2,
                       Y0_2,
                       net, 
                       device,
                       False)

/tmp/dcs-tmp.u1834031/ipykernel_2033954/372614191.py:26: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  self.u_weights = torch.tensor(u_weights, requires_grad=True)


In [11]:
print(model.u_weights)

tensor([[2.2975e-01],
        [2.3291e-01],
        [6.1409e-02],
        [5.5300e-01],
        [6.2458e-01],
        [2.7098e-02],
        [3.3405e-01],
        [2.4458e-01],
        [7.6559e-01],
        [1.3041e-01],
        [4.0268e-01],
        [5.5725e-01],
        [4.5174e-01],
        [5.7326e-01],
        [9.4319e-01],
        [1.8388e-01],
        [3.9757e-01],
        [6.9500e-01],
        [7.6124e-01],
        [7.4316e-01],
        [4.7308e-01],
        [1.8787e-01],
        [2.0427e-01],
        [3.8139e-01],
        [6.8115e-01],
        [5.0061e-01],
        [6.6192e-01],
        [7.0493e-01],
        [8.0040e-01],
        [3.8677e-01],
        [5.1218e-02],
        [8.7955e-01],
        [9.1547e-02],
        [8.9078e-01],
        [1.1143e-01],
        [8.1999e-01],
        [4.6197e-01],
        [3.5769e-01],
        [9.5017e-02],
        [8.4051e-01],
        [1.4791e-01],
        [6.8505e-01],
        [2.6940e-01],
        [3.5625e-01],
        [2.1081e-01],
        [5

In [18]:
%%time
model.train(phase_1=False,epoch=100,X = X_star,U = u_star)

/tmp/dcs-tmp.u1834031/ipykernel_2033954/372614191.py:64: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  x = torch.tensor(X[:, 0:1], requires_grad=True).float().to(self.device)
/tmp/dcs-tmp.u1834031/ipykernel_2033954/372614191.py:65: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  t = torch.tensor(X[:, 1:2], requires_grad=True).float().to(self.device)
/tmp/dcs-tmp.u1834031/ipykernel_2033954/372614191.py:77: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  x_lb = torch.tensor(self.train_x_lb[:, 0:1], requires_grad=True).float().to(self.device)
/t

Iter 34700, Error u: 4.524378e-01
Iter 34800, Error u: 3.497594e-01
Iter 34900, Error u: 1.171693e-01
Iter 35000, Error u: 5.922866e-02
Iter 35100, Error u: 5.382470e-02
Iter 35200, Error u: 4.651323e-02
Iter 35300, Error u: 4.157838e-02
Iter 35400, Error u: 3.912363e-02
Iter 35500, Error u: 3.757090e-02
Iter 35600, Error u: 3.496992e-02
Iter 35700, Error u: 3.399221e-02
Iter 35800, Error u: 3.363373e-02
Iter 35900, Error u: 3.364574e-02
Iter 36000, Error u: 3.306709e-02
Iter 36100, Error u: 3.259311e-02
Iter 36200, Error u: 3.131451e-02
Iter 36300, Error u: 3.031929e-02
Iter 36400, Error u: 2.927613e-02
Iter 36500, Error u: 2.894340e-02
Iter 36600, Error u: 2.831306e-02
Iter 36700, Error u: 2.737892e-02
Iter 36800, Error u: 2.678122e-02
Iter 36900, Error u: 2.637024e-02
Iter 37000, Error u: 2.558490e-02
Iter 37100, Error u: 2.491200e-02
Iter 37200, Error u: 2.446277e-02
Iter 37300, Error u: 2.368294e-02
Iter 37400, Error u: 2.231208e-02
Iter 37500, Error u: 2.119868e-02
Iter 37600, Er

In [15]:
print(model.u_weights)

tensor([[ 1.6050e+01],
        [ 6.0601e+00],
        [ 9.9895e-01],
        [ 1.8152e+02],
        [ 1.0021e+00],
        [ 9.9112e-01],
        [ 6.5414e+00],
        [ 1.0014e+00],
        [ 1.0972e+01],
        [ 1.0001e+00],
        [ 6.4780e+00],
        [ 9.1762e-01],
        [ 3.9447e+00],
        [ 9.9799e-01],
        [ 9.9954e-01],
        [ 1.1053e+00],
        [-2.9686e+01],
        [ 9.9944e-01],
        [ 9.9983e-01],
        [ 1.0010e+00],
        [-3.4591e+01],
        [ 9.9865e-01],
        [-3.4826e+01],
        [ 1.0026e+00],
        [ 2.7603e+00],
        [ 1.0026e+00],
        [ 1.9325e+00],
        [ 1.0007e+00],
        [-1.6530e+00],
        [-2.1782e+01],
        [ 5.5476e+00],
        [ 1.0008e+00],
        [ 9.9379e-01],
        [ 9.9485e-01],
        [ 8.0904e-01],
        [ 4.3714e+01],
        [ 6.6470e+00],
        [-2.6559e+01],
        [-3.2451e+01],
        [ 9.9197e-01],
        [ 9.9556e-01],
        [-2.2526e+01],
        [ 9.9920e-01],
        [ 1

In [16]:
u_pred, f_pred = model.predict(X_star)

error_u = np.linalg.norm(u_star-u_pred,2)/np.linalg.norm(u_star,2)
print('Error u: %e' % (error_u))
print('Residual: %e' % (f_pred**2).mean())

U_pred = griddata(X_star, u_pred.flatten(), (X, T), method='cubic')
Error = np.abs(Exact.T - U_pred)

Error u: 5.128660e-01
Residual: 5.533923e-03


In [13]:
U_pred = griddata(X_star, u_pred.flatten(), (X, T), method='cubic')
Error = np.abs(Exact.T - U_pred)
""" The aesthetic setting has changed. """


xx1 = np.hstack((X[0:1,:].T, T[0:1,:].T))
uu1 = Exact[0:1,:].T

# boundary conditions x = lb
xx2 = np.hstack((X[:,0:1], T[:,0:1]))
uu2 = Exact[:,0:1]

# boundary conditions, x = ub
xx3 = np.hstack((X[:,-1:], T[:,-1:]))
uu3 = Exact[:,-1:]

X_u_train = np.vstack([xx2, xx3]) 


####### Row 0: u(t,x) ##################    
X_u_train_ = X_u_train

####### Row 0: u(t,x) ##################    

fig = plt.figure(figsize=(9, 5))
ax = fig.add_subplot(111)
t = data['tt'].flatten()[:,None]
x = data['x'].flatten()[:,None]

h = ax.imshow(Error.T, interpolation='nearest', cmap='rainbow', 
              extent=[t.min(), t.max(), x.min(), x.max()], 
              origin='lower', aspect='auto')
            #   vmin = 0,
            #   vmax = 0.5770284271569969)
divider = make_axes_locatable(ax)
cax = divider.append_axes("right", size="5%", pad=0.10)
cbar = fig.colorbar(h, cax=cax)
cbar.ax.tick_params(labelsize=15) 

# ax.plot(
#     X_u_train_[:,1], 
#     X_u_train_[:,0], 
#     markersize = 4,  # marker size doubled
#     clip_on = False,
#     alpha=1.0
# )


ax.set_xlabel('$t$', size=20)
ax.set_ylabel('$x$', size=20)
# ax.legend(
#     loc='upper center', 
#     bbox_to_anchor=(0.9, -0.05), 
#     ncol=5, 
#     frameon=False, 
#     prop={'size': 15}
# )
ax.set_title('Error $u(x,t)$', fontsize = 20) # font size doubled
ax.tick_params(labelsize=15)


plt.show()

KeyboardInterrupt: 